In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
np.set_printoptions(precision=8)

In [3]:
path="/Users/ecasiano/Desktop/PrototypeScripts/TestSystematicError/"

In [4]:
seed = 42

In [5]:
data_Pn_squared_m1 = np.loadtxt(path+"1D_4_4_2_3.300000_1.000000_4.000000_100000_PnSquared-mA1_"+str(seed)+".dat")
data_Pn_squared_m2 = np.loadtxt(path+"1D_4_4_2_3.300000_1.000000_4.000000_100000_PnSquared-mA2_"+str(seed)+".dat")

data_SWAPn_m1 = np.loadtxt(path+"1D_4_4_2_3.300000_1.000000_4.000000_100000_SWAPn-mA1_"+str(seed)+".dat")
data_SWAPn_m2 = np.loadtxt(path+"1D_4_4_2_3.300000_1.000000_4.000000_100000_SWAPn-mA2_"+str(seed)+".dat")

In [6]:
Pn_squared_m1_col_sum = np.sum(data_Pn_squared_m1,axis=0)
Pn_squared_m2_col_sum = np.sum(data_Pn_squared_m2,axis=0)

SWAPn_m1_col_sum = np.sum(data_SWAPn_m1,axis=0)
SWAPn_m2_col_sum = np.sum(data_SWAPn_m2,axis=0)

In [7]:
SWAPn_m2_col_sum.astype(int),Pn_squared_m2_col_sum.astype(int)

(array([    49881,  19659001, 183754485,  19765536,     45433]),
 array([    49725,  20002866, 184948149,  20091390,     51267]))

In [8]:
-np.log(SWAPn_m2_col_sum.astype(int)/Pn_squared_m2_col_sum.astype(int))

array([-0.00313234,  0.01734026,  0.00647496,  0.01635155,  0.12080856])

In [9]:
SWAPn_m2_col_sum[0]-SWAPn_m2_col_sum[-1],Pn_squared_m2_col_sum[0]-Pn_squared_m2_col_sum[-1]

(4448.0, -1542.0)

In [10]:
# -np.log(np.mean(data_SWAPn_m2,axis=0)/np.mean(data_Pn_squared_m2,axis=0))
# -np.log(np.mean(data_SWAPn_m2,axis=0)/np.mean(data_Pn_squared_m2,axis=0))

### Jacknife notes

In [11]:
N=data_SWAPn_m1.shape[1]-1
N

4

In [12]:
S2 = np.zeros(data_SWAPn_m1.shape)
for i in range(data_SWAPn_m2.shape[0]):
    Pn_squared_m1_col_sum_i = Pn_squared_m1_col_sum - data_Pn_squared_m1[i]
    Pn_squared_m2_col_sum_i = Pn_squared_m2_col_sum - data_Pn_squared_m2[i]
    
    SWAPn_m1_col_sum_i = SWAPn_m1_col_sum - data_SWAPn_m1[i]
    SWAPn_m2_col_sum_i = SWAPn_m2_col_sum - data_SWAPn_m2[i]
    
    S2[i] = -np.log(SWAPn_m2_col_sum_i/Pn_squared_m2_col_sum_i)
S2

array([[-0.00313234,  0.01734085,  0.00647969,  0.0163482 ,  0.12080856],
       [-0.00313234,  0.01733539,  0.00647269,  0.01635198,  0.12080856],
       [-0.00313234,  0.01734078,  0.00647271,  0.01634824,  0.12080856],
       ...,
       [-0.00313234,  0.01731607,  0.00647216,  0.01635102,  0.12080856],
       [-0.00313234,  0.01735643,  0.00646946,  0.01634865,  0.12080856],
       [-0.00313234,  0.01732955,  0.00648027,  0.01635155,  0.12080856]])

In [13]:
S2_mean = np.mean(S2,axis=0)
S2_mean

array([-0.00313234,  0.01734026,  0.00647496,  0.01635155,  0.12080857])

In [14]:
N_rows = S2.shape[0]

In [15]:
S2_err = np.std(S2,axis=0)
S2_err *= np.sqrt(N_rows-1)

In [16]:
for n in range(N+1):
    print(f"<S2(n={n})> = {S2_mean[n]:0.6f} +/- {S2_err[n]:0.6f}")

<S2(n=0)> = -0.003132 +/- 0.006364
<S2(n=1)> = 0.017340 +/- 0.003850
<S2(n=2)> = 0.006475 +/- 0.001251
<S2(n=3)> = 0.016352 +/- 0.004285
<S2(n=4)> = 0.120809 +/- 0.080557


### $P(n)$

In [19]:
data_Pn_m2 = np.loadtxt(path+"1D_4_4_2_3.300000_1.000000_4.000000_100000_Pn-mA2_"+str(seed)+".dat")
np.mean(data_Pn_m2,axis=0)

array([ 101.77536, 2057.49176, 6256.24106, 2061.23979,  102.57595])

In [20]:
data_Pn_col_sum_m2 = np.sum(data_Pn_m2,axis=0)

In [21]:
jacknifed_Pn = np.zeros(data_Pn_m2.shape)
for i in range(data_Pn_m2.shape[0]):
    data_Pn_col_sum_m2_i = data_Pn_col_sum_m2 - data_Pn_m2[i]
        
    jacknifed_Pn[i] = data_Pn_col_sum_m2_i
jacknifed_Pn /= np.sum(jacknifed_Pn,axis=1)[:,None]

In [22]:
Pn_mean = np.mean(jacknifed_Pn,axis=0)
Pn_mean /= np.sum(Pn_mean )
N_rows_Pn = jacknifed_Pn.shape[0]
Pn_err = np.std(jacknifed_Pn,axis=0)
Pn_err *= np.sqrt(N_rows_Pn-1)

In [23]:
for n in range(4+1):
    print(f'P(n={n}) = {Pn_mean[n]:0.4f} +/- {Pn_err[n]:0.4f}')

P(n=0) = 0.0096 +/- 0.0001
P(n=1) = 0.1945 +/- 0.0002
P(n=2) = 0.5914 +/- 0.0002
P(n=3) = 0.1948 +/- 0.0002
P(n=4) = 0.0097 +/- 0.0001
